# ARIMA

In [1]:
import pandas as pd
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np
from pandas.tools.plotting import autocorrelation_plot

/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [69]:
def mape(y_p, y):
    y[y == 0] = 0.0001
    diff = np.abs((y_p - y) / y)
    #print(diff)
    return 100. * np.mean(diff)

# RMSE loss for calculate test result.
def rmse(y_p, y):
    diff = np.sqrt((y_p - y)**2)# np.sum()
    return np.mean(diff)


def ARIMA_performance(df,daterange):
    #print(df['FishnetID'])
    ts_cur = pd.Series(df['PerformanceTarget'].values, index=daterange)
    X = ts_cur.fillna(0).values
    y = ts_cur[3:]
    train, test = X[0:12], X[12:]
    #train = X[0:9]
    history = [x for x in train]
    predictions = list()
    for t in range(len(test)):
        try:
            model = ARIMA(history, order=(3,0,3))
            model_fit = model.fit(disp=-1, method='mle')
            output = model_fit.predict()
            #print(output[::-1])
            #print(y)
            y_pred = output[::-1][2:]
            mape_cur = mape(y_pred,y)
            rmse_cur = rmse(y_pred,y)
        except:
            mape_cur = 0
            rmse_cur = 0
    return(mape_cur, rmse_cur)
        

## Hospitality

In [17]:
hos_df = pd.read_csv('3_hospitality_13months.csv').iloc[:, 1:8]
print(hos_df.shape[0] /13)
hos_df = hos_df[['FishnetID','ReportingMonth','PerformanceTarget']]

343.0


In [18]:
hos_df.fillna(0)
hos_df.head()

,FishnetID,ReportingMonth,PerformanceTarget
0,124679,2017-06-01,0.9167
1,124679,2017-07-01,1.0000
2,124679,2017-08-01,1.0000
3,124679,2017-09-01,1.0000
4,124679,2017-10-01,1.0000


In [19]:
hos_df['ReportingMonth'] = pd.to_datetime(hos_df['ReportingMonth'])

In [20]:
dr = pd.date_range('2017-6', periods=13, freq='M')

In [47]:
fids = hos_df['FishnetID'].unique().tolist()

In [70]:
mape_overall = []
rmse_overall = []
for fid in fids:
    df_cur = hos_df[hos_df['FishnetID'] == fid]
    mape_c, rmse_c = ARIMA_performance(df_cur, dr)
    mape_overall.append(mape_c)
    rmse_overall.append(rmse_c)

/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/ar_model.py:770: RuntimeWarning: divide by zero encountered in log
  return np.log(self.sigma2) + (1 + self.df_model) * np.log(nobs)/nobs
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:646: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if issubdtype(paramsdtype, float):
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:650: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  elif issubdtype(paramsdtype, complex):
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hess

/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning

/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning

In [82]:
mape_overall = np.array(mape_overall)
mape_overall_filter = mape_overall[(mape_overall < 30 ) & (mape_overall > 0 )]
np.average(mape_overall_filter)

5.089061481945484

In [83]:
rmse_overall = np.array(rmse_overall)
rmse_overall_filter = rmse_overall[(mape_overall < 30 ) & (mape_overall > 0 )]
np.average(rmse_overall_filter)

0.04335903029803879

## Retail

In [87]:
ret_df = pd.read_csv('4_retail_13months.csv').iloc[:, 1:8]
ret_df.shape
ret_df['ReportingMonth'] = pd.to_datetime(ret_df['ReportingMonth'])
dr = pd.date_range('2017-6', periods=13, freq='M')
fids = ret_df['FishnetID'].unique().tolist()

In [88]:
ret_df = ret_df[['FishnetID','ReportingMonth','PerformanceTarget']]
ret_df.fillna(0)
ret_df.head()

,FishnetID,ReportingMonth,PerformanceTarget
0,144322,2017-06-01,1.0000
1,144322,2017-07-01,1.0000
2,144322,2017-08-01,1.0000
3,144322,2017-09-01,1.0000
4,144322,2017-10-01,0.9732


In [89]:
mape_overall_ret = []
rmse_overall_ret = []
for fid in fids:
    df_cur = ret_df[ret_df['FishnetID'] == fid]
    mape_c, rmse_c = ARIMA_performance(df_cur, dr)
    mape_overall_ret.append(mape_c)
    rmse_overall_ret.append(rmse_c)

/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:646: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if issubdtype(paramsdtype, float):
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:650: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  elif issubdtype(paramsdtype, complex):
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood o

/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning

/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning

In [90]:
mape_overall_ret = np.array(mape_overall_ret)
mape_overall_ret_filter = mape_overall_ret[(mape_overall_ret < 30 ) & (mape_overall_ret > 0 )]
np.average(mape_overall_ret_filter)

5.050060706684591

In [91]:
rmse_overall_ret = np.array(rmse_overall_ret)
rmse_overall_ret_filter = rmse_overall_ret[(rmse_overall_ret < 30 ) & (rmse_overall_ret > 0 )]
np.average(rmse_overall_ret_filter)

0.057281865597373735

## Building

In [92]:
bud_df = pd.read_csv('1_building_13months.csv').iloc[:, 1:8]
bud_df.shape
bud_df['ReportingMonth'] = pd.to_datetime(bud_df['ReportingMonth'])
dr = pd.date_range('2017-6', periods=13, freq='M')
fids = bud_df['FishnetID'].unique().tolist()

In [93]:
bud_df = bud_df[['FishnetID','ReportingMonth','PerformanceTarget']]
bud_df.fillna(0)
bud_df.head()

,FishnetID,ReportingMonth,PerformanceTarget
0,127137,2017-06-01,0.9415
1,127137,2017-07-01,0.9657
2,127137,2017-08-01,0.9869
3,127137,2017-09-01,0.9913
4,127137,2017-10-01,0.9897


In [95]:
mape_overall_bud = []
rmse_overall_bud = []
for fid in fids:
    df_cur = bud_df[bud_df['FishnetID'] == fid]
    mape_c, rmse_c = ARIMA_performance(df_cur, dr)
    mape_overall_bud.append(mape_c)
    rmse_overall_bud.append(rmse_c)

/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:646: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if issubdtype(paramsdtype, float):
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:650: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  elif issubdtype(paramsdtype, complex):
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/kalmanf/kalmanfilter.py:577: FutureWarning: 

/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning

/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/michael.zhang/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning

In [96]:
mape_overall_bud = np.array(mape_overall_bud)
mape_overall_bud_filter = mape_overall_bud[(mape_overall_bud < 30 ) & (mape_overall_bud > 0 )]
np.average(mape_overall_bud_filter)

6.309159910979936

In [97]:
rmse_overall_bud = np.array(rmse_overall_bud)
rmse_overall_bud_filter = rmse_overall_bud[(rmse_overall_bud < 30 ) & (rmse_overall_bud > 0 )]
np.average(rmse_overall_bud_filter)

0.08535319798006502

In [98]:
k = np.random.rand(3,2,2,2)

In [104]:
k[0]

array([[[0.09724746, 0.25207021],
        [0.66664426, 0.42216634]],

       [[0.96251081, 0.54836979],
        [0.56428619, 0.52345494]]])

In [102]:
k_r = k.reshape(3,8)

In [105]:
k_r[0]

array([0.09724746, 0.25207021, 0.66664426, 0.42216634, 0.96251081,
       0.54836979, 0.56428619, 0.52345494])